In [6]:
import pandas as pd
import wrds

In [7]:
conn = wrds.Connection(wrds_username='yjzhu')
conn.create_pgpass_file() 

Loading library list...
Done
Enter your WRDS username [admin]:yjzhu
Enter your password:········


In [8]:
import numpy as np
from scipy.stats.mstats import winsorize
from scipy import stats
import statsmodels.formula.api as sm
import patsy
import statsmodels.api as sm
from statsmodels.formula.api import ols

In [9]:
#Sample Selection
funda = conn.raw_sql("""select exchg, gvkey, fyear,datadate, oiadp,ni, csho, at,act,che,lct,dlc,txp,dp,prcc_f,sich,dvt,lt,bkvlps,acqinvt,mkvalt    
                         from compa.funda 
                         where consol='C' and indfmt='INDL' and datafmt='STD' and popsrc='D' and  fyear<2020 and fyear>2016
                         """) 
company = conn.raw_sql("""select gvkey, sic,conml,spcsrc
                          from compa.company
                          """) 
funda1=pd.merge(funda,company,on=['gvkey'])
funda1['sic1']=np.where(funda1['sich']>0,funda1['sich'],funda1['sic'])
funda1=funda1.drop(['sich','sic'],axis=1)
funda1['sic1']=funda1['sic1'].astype(str)

In [10]:
funda1

,exchg,gvkey,fyear,datadate,oiadp,ni,csho,at,act,che,...,dp,prcc_f,dvt,lt,bkvlps,acqinvt,mkvalt,conml,spcsrc,sic1
0,11.0,001004,2017.0,2018-05-31,86.000,15.600,34.716,1524.700,942.700,41.600,...,40.500,44.6900,10.300,588.400,26.9703,NaN,1551.4580,AAR Corp,B,5080.0
1,11.0,001119,2017.0,2017-12-31,NaN,NaN,98.685,NaN,NaN,NaN,...,NaN,15.0300,NaN,NaN,NaN,NaN,NaN,Adams Diversified Equity Fund Inc,None,6726
2,11.0,001119,2018.0,2018-12-31,NaN,NaN,100.868,NaN,NaN,NaN,...,NaN,12.6200,NaN,NaN,NaN,NaN,NaN,Adams Diversified Equity Fund Inc,None,6726
3,14.0,001045,2017.0,2017-12-31,4792.000,1919.000,475.508,51396.000,9146.000,5384.000,...,2017.000,52.0300,198.000,47470.000,8.2564,NaN,24740.6812,American Airlines Group Inc,B-,4512.0
4,14.0,001045,2018.0,2018-12-31,3449.000,1412.000,460.611,60580.000,8637.000,4914.000,...,2157.000,32.1100,188.000,60749.000,-0.3669,NaN,14790.2192,American Airlines Group Inc,B-,4512.0
5,14.0,001050,2017.0,2017-12-31,11.977,-3.029,34.570,438.549,173.942,30.493,...,16.088,5.1300,7.792,251.980,5.3968,NaN,177.3441,CECO Environmental Corp.,C,3564.0
6,14.0,001050,2018.0,2018-12-31,14.062,-7.121,34.816,392.582,160.865,44.438,...,13.272,6.7500,0.000,214.022,5.1287,NaN,235.0080,CECO Environmental Corp.,C,3564.0
7,17.0,001352,2017.0,2017-12-31,NaN,NaN,4.875,NaN,NaN,NaN,...,NaN,17.9900,NaN,NaN,NaN,NaN,NaN,Arrow Dow Jones Global Yield ETF,None,6722
8,17.0,001352,2018.0,2018-12-31,NaN,NaN,3.750,NaN,NaN,NaN,...,NaN,15.0682,NaN,NaN,NaN,NaN,NaN,Arrow Dow Jones Global Yield ETF,None,6722
9,11.0,001062,2017.0,2017-11-30,-1.670,1.744,19.290,245.562,NaN,3.678,...,0.003,11.0526,0.772,1.360,12.6595,NaN,213.2047,ASA Gold and Precious Metals Ltd,None,6799.0


In [11]:
#Generate Lag Variable
funda1_lag1 = funda1[['gvkey','fyear','at','act','che','lct','dlc','txp']].copy()
funda1_lag1['fyear'] = funda1_lag1['fyear']+1
funda1_lag1=funda1_lag1.rename(columns = {'at':'at_lag1','act':'act_lag1','che':'che_lag1','lct':'lct_lag1','dlc':'dlc_lag1',
                                       'txp':'txp_lag1'})
funda1=pd.merge(funda1, funda1_lag1, how = 'left', on = ['gvkey','fyear'])

In [12]:
#winsorize the variable
varlist  = ['at','at_lag1','act_lag1','che_lag1','lct_lag1','dlc_lag1','txp_lag1','act','che','lct','dlc','txp','oiadp','ni','csho','dp','dvt','lt','bkvlps']
for var in varlist:    
    funda1[var] = funda1[var].replace(np.Inf, np.nan)    
    funda1[var]=np.where(funda1[var].isnull(), np.nan, winsorize(funda1[var], limits=(0.01,0.01)))

In [13]:
#Construct other Variables
funda1['AverageTotalAsset'] = (funda1['at']+funda1['at_lag1'])/2
funda1['Earnings'] = funda1['oiadp']/funda1['AverageTotalAsset']
funda1['chg_ca']=funda1['act']-funda1['act_lag1']
funda1['chg_cash'] = funda1['che']-funda1['che_lag1']
funda1['chg_cl'] =funda1['lct']-funda1['lct_lag1']
funda1['chg_std'] = funda1['dlc']-funda1['dlc_lag1']
funda1['chg_tp'] = funda1['txp']-funda1['txp_lag1']
funda1['Accural_Comp'] = ((funda1['chg_ca']-funda1['chg_cash']) -(funda1['chg_cl']-funda1['chg_std']-funda1['chg_tp'])-funda1['dp'])/funda1['AverageTotalAsset']
funda1['be'] = funda1['at']-funda1['lt']
funda1['CashFlow']=funda1['Earnings']-funda1['Accural_Comp']

In [14]:
funda1.describe()

,exchg,fyear,oiadp,ni,csho,at,act,che,lct,dlc,...,AverageTotalAsset,Earnings,chg_ca,chg_cash,chg_cl,chg_std,chg_tp,Accural_Comp,be,CashFlow
count,21090.000000,21094.000000,14989.000000,14990.000000,20851.000000,1.506800e+04,12379.000000,15067.000000,12397.000000,15061.000000,...,6.975000e+03,6969.000000,5692.000000,6974.000000,5698.000000,6971.000000,6250.000000,5496.000000,15054.000000,5496.000000
mean,14.225036,2017.484451,554.750381,317.109993,160.217897,1.498620e+04,1658.829639,1735.889714,1344.417758,937.577989,...,1.500513e+04,-0.200587,36.017704,-36.136371,69.114505,-1.386446,-0.391624,-0.036672,2840.330100,-0.229401
std,4.056467,0.503173,2891.509348,1667.434235,838.002718,1.090169e+05,7719.271129,18622.483756,6608.291231,10113.257866,...,1.052209e+05,1.979958,1325.322745,3507.795961,1217.709106,1958.851400,84.500665,1.801008,13025.741297,2.667076
min,0.000000,2017.000000,-174.684000,-418.258000,0.100000,5.900000e-02,0.021000,0.000000,0.086000,0.000000,...,2.215000e-01,-78.960875,-29578.000000,-76034.839000,-19431.000000,-45170.000000,-1664.000000,-52.659142,-62637.000000,-107.620767
25%,11.000000,2017.000000,-3.345000,-8.057750,7.702000,4.945175e+01,10.970000,4.971500,5.290000,0.000000,...,6.169800e+01,-0.102948,-8.639750,-13.577750,-2.615250,-1.142500,0.000000,-0.075479,16.947000,-0.177095
50%,14.000000,2017.000000,12.210000,2.849500,31.564000,5.277150e+02,114.428000,41.338000,52.341000,2.330000,...,6.039340e+02,0.023094,0.648000,-0.012500,0.894000,0.000000,0.000000,-0.033838,175.588000,0.057987
75%,17.000000,2018.000000,183.898000,94.845000,93.989500,3.263210e+03,635.650000,219.941000,400.339000,50.000000,...,3.583393e+03,0.074642,40.078250,15.226250,22.926250,2.318500,0.000000,0.000871,1082.456750,0.131256
max,22.000000,2019.000000,115076.000000,59531.000000,66623.515000,3.418318e+06,210129.924000,768210.644000,192819.656000,327848.000000,...,3.381924e+06,110.864415,42336.682000,179161.593000,34566.000000,65172.000000,3034.360000,105.255079,352500.000000,110.974058


In [15]:
fundaclean1 = funda1[['gvkey','spcsrc','fyear','Earnings','Accural_Comp','at','dvt','bkvlps','acqinvt','conml']].copy()

In [16]:
fundaclean1['dvt'] = fundaclean1['dvt'].replace(np.nan, 0)
fundaclean1['DD'] = np.where(fundaclean1['dvt']==0,0,1)
fundaclean1['Neg_E'] = np.where(fundaclean1['Earnings']<0,1,0)
#fundaclean1['acqinvt'] = fundaclean1['acqinvt'].replace(np.nan, 0)
#fundaclean1['inv_dummy'] = np.where(fundaclean1['dvt']==0,0,1)

In [17]:
fundaclean1['bkvlps'] = fundaclean1['bkvlps'].replace(np.nan, 0)

In [18]:
fundaclean1['acqinvt'] = fundaclean1['acqinvt'].replace(np.nan, 0)

In [19]:
fundaclean2 = fundaclean1[fundaclean1['fyear'] == 2018]

In [20]:
fundaclean2.describe()

,fyear,Earnings,Accural_Comp,at,dvt,bkvlps,acqinvt,DD,Neg_E
count,10147.0,6969.000000,5496.000000,7.059000e+03,10147.000000,1.014700e+04,10147.000000,10147.000000,10147.000000
mean,2018.0,-0.200587,-0.036672,1.489558e+04,108.832588,9.772483e+03,5.520863,0.301370,0.259978
std,0.0,1.979958,1.801008,1.044352e+05,669.558123,3.372892e+05,116.503747,0.458876,0.438644
min,2018.0,-78.960875,-52.659142,5.900000e-02,0.000000,-1.019140e+01,0.000000,0.000000,0.000000
25%,2018.0,-0.102948,-0.075479,6.169200e+01,0.000000,0.000000e+00,0.000000,0.000000,0.000000
50%,2018.0,0.023094,-0.033838,6.221040e+02,0.000000,6.467000e-01,0.000000,0.000000,0.000000
75%,2018.0,0.074642,0.000871,3.639062e+03,3.908000,1.051410e+01,0.000000,1.000000,1.000000
max,2018.0,110.864415,105.255079,3.418318e+06,18849.000000,2.101400e+07,6239.147000,1.000000,1.000000


In [21]:
fundaclean2 = fundaclean2.dropna()

In [22]:
fundaclean2.describe()

,fyear,Earnings,Accural_Comp,at,dvt,bkvlps,acqinvt,DD,Neg_E
count,2518.0,2518.000000,2518.000000,2518.000000,2518.000000,2.518000e+03,2518.000000,2518.000000,2518.000000
mean,2018.0,-0.107726,-0.058106,6734.497155,166.685224,5.552261e+03,8.255227,0.464257,0.318904
std,0.0,1.015236,1.094680,24148.431480,831.248213,2.137331e+05,96.873893,0.498820,0.466144
min,2018.0,-34.306998,-52.659142,0.059000,0.000000,-1.019140e+01,0.000000,0.000000,0.000000
25%,2018.0,-0.036042,-0.069131,79.711000,0.000000,8.496750e-01,0.000000,0.000000,0.000000
50%,2018.0,0.054329,-0.033974,709.670000,0.000000,6.631000e+00,0.000000,0.000000,0.000000
75%,2018.0,0.106036,-0.001902,3569.200000,29.749000,1.779478e+01,0.000000,1.000000,1.000000
max,2018.0,2.040343,8.076749,531864.000000,14117.000000,9.957000e+06,3303.000000,1.000000,1.000000


In [23]:
A = ['A+','A','A-']
B = ['B+','B','B-']
fundaclean2['stock_rate'] = np.where(fundaclean2['spcsrc'].isin(A), 'A', 
                             (np.where(fundaclean2['spcsrc'].isin(B),'B','others')))

In [24]:
fundaclean2 = pd.get_dummies(fundaclean2,columns=['stock_rate'])

In [25]:
fundaclean2=fundaclean2.rename(columns={'dvt':'D','bkvlps':'B','Earnings':'E','Accural_Comp':'AC','at':'A'})

In [26]:
fundaclean2.describe()

,fyear,E,AC,A,D,B,acqinvt,DD,Neg_E,stock_rate_A,stock_rate_B,stock_rate_others
count,2518.0,2518.000000,2518.000000,2518.000000,2518.000000,2.518000e+03,2518.000000,2518.000000,2518.000000,2518.000000,2518.000000,2518.000000
mean,2018.0,-0.107726,-0.058106,6734.497155,166.685224,5.552261e+03,8.255227,0.464257,0.318904,0.098888,0.511914,0.389198
std,0.0,1.015236,1.094680,24148.431480,831.248213,2.137331e+05,96.873893,0.498820,0.466144,0.298571,0.499957,0.487665
min,2018.0,-34.306998,-52.659142,0.059000,0.000000,-1.019140e+01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2018.0,-0.036042,-0.069131,79.711000,0.000000,8.496750e-01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,2018.0,0.054329,-0.033974,709.670000,0.000000,6.631000e+00,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
75%,2018.0,0.106036,-0.001902,3569.200000,29.749000,1.779478e+01,0.000000,1.000000,1.000000,0.000000,1.000000,1.000000
max,2018.0,2.040343,8.076749,531864.000000,14117.000000,9.957000e+06,3303.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [27]:
fundaclean2['predict_2019'] = 0.0114+0.7280*fundaclean2['E']-0.0047*fundaclean2['Neg_E']-0.0106*fundaclean2['Neg_E']*fundaclean2['E']-0.0001*fundaclean2['B']+0.0369*fundaclean2['AC']+0.0240*fundaclean2['stock_rate_A']+0.0091*fundaclean2['stock_rate_B']-0.0217*fundaclean2['stock_rate_others']+0*fundaclean2['acqinvt']

In [28]:
fundaclean2.to_csv(r"C:/Users/admin/Desktop/Q4_prediction.csv")